<a href="https://colab.research.google.com/github/AkhilVinayakp/HAR_LSTM_CNN/blob/main/inp_pipe/inp_pipeline_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from statistics import mode

In [59]:
%cd /content/drive/My\ Drive/datasets 
%pwd

/content/drive/My Drive/datasets


'/content/drive/My Drive/datasets'

In [61]:

df =  pd.read_csv('inter_norm_mini.csv')

In [ ]:
df.head()

,user,activity,timestamp,x-axis,y-axis,z-axis
0,14,Downstairs,57725432324000,0.668528,0.725448,-0.163691
1,7,Upstairs,208866321473000,0.454967,0.875538,-0.162600
2,29,Jogging,0,0.287281,0.887311,-0.360761
3,15,Jogging,0,0.944283,0.292972,-0.149990
4,8,Jogging,111020251899000,-0.124547,-0.706035,-0.697139


In [62]:
df.drop('user', inplace=True, axis=1)

In [63]:
df.drop('timestamp', inplace=True, axis=1)

In [ ]:
df

,activity,x-axis,y-axis,z-axis
0,Downstairs,0.668528,0.725448,-0.163691
1,Upstairs,0.454967,0.875538,-0.162600
2,Jogging,0.287281,0.887311,-0.360761
3,Jogging,0.944283,0.292972,-0.149990
4,Jogging,-0.124547,-0.706035,-0.697139
...,...,...,...,...
105,Downstairs,0.449064,0.893440,-0.010313
106,Downstairs,-0.157978,0.929553,0.333128
107,Jogging,0.687072,0.721425,-0.086476
108,Walking,0.927688,0.373049,-0.015133


In [64]:
df.activity = df.activity.astype('category')
df.activity = df.activity.cat.codes.astype('float64')

In [ ]:
df

,activity,x-axis,y-axis,z-axis
0,0.0,0.668528,0.725448,-0.163691
1,4.0,0.454967,0.875538,-0.162600
2,1.0,0.287281,0.887311,-0.360761
3,1.0,0.944283,0.292972,-0.149990
4,1.0,-0.124547,-0.706035,-0.697139
...,...,...,...,...
105,0.0,0.449064,0.893440,-0.010313
106,0.0,-0.157978,0.929553,0.333128
107,1.0,0.687072,0.721425,-0.086476
108,5.0,0.927688,0.373049,-0.015133


In [ ]:
df.dtypes

activity    float64
x-axis      float64
y-axis      float64
z-axis      float64
dtype: object

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(df)

In [ ]:
window_size = 20

In [ ]:
def enc(y):
  return y+1

In [65]:
# dataset = dataset.batch(20, drop_remainder=True)
dataset = tf.data.Dataset.from_tensor_slices(df)
dataset = dataset.window(20,shift=10, drop_remainder=True)
dataset = dataset.flat_map(lambda window : window.batch(20))
dataset = dataset.map(lambda window: (window[:,1:],window[:,0]))
dataset = dataset.map(lambda: enc(x))

TypeError: ignored

In [ ]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [ ]:
for window in dataset:
  print(window.numpy())

AttributeError: ignored

In [82]:
def encod(window):
  (x, y) = window
  y = y + 1
  print(y)
  return (x, y) 



In [101]:
@tf.function
def enc(window):
  r = window.numpy()
  return mode(r)


In [84]:
# test datset
sr = np.arange(0,50)
sr = sr.reshape(10,5)

In [109]:
sr_set = tf.data.Dataset.from_tensor_slices(sr)
sr_set = sr_set.window(4,shift=2, drop_remainder=True)
sr_set = sr_set.flat_map(lambda window : window.batch(4))
sr_set = sr_set.map(lambda window: (window[:,1:],window[3,0]))
# sr_set = sr_set.map(lambda x: encod(x))

In [110]:
for x, y in sr_set:
  print('x=', x)
  print('y =', y)

x= tf.Tensor(
[[ 1  2  3  4]
 [ 6  7  8  9]
 [11 12 13 14]
 [16 17 18 19]], shape=(4, 4), dtype=int64)
y = tf.Tensor(15, shape=(), dtype=int64)
x= tf.Tensor(
[[11 12 13 14]
 [16 17 18 19]
 [21 22 23 24]
 [26 27 28 29]], shape=(4, 4), dtype=int64)
y = tf.Tensor(25, shape=(), dtype=int64)
x= tf.Tensor(
[[21 22 23 24]
 [26 27 28 29]
 [31 32 33 34]
 [36 37 38 39]], shape=(4, 4), dtype=int64)
y = tf.Tensor(35, shape=(), dtype=int64)
x= tf.Tensor(
[[31 32 33 34]
 [36 37 38 39]
 [41 42 43 44]
 [46 47 48 49]], shape=(4, 4), dtype=int64)
y = tf.Tensor(45, shape=(), dtype=int64)


In [113]:
t = sr_set.as_numpy_iterator()

TypeError: ignored

In [114]:
r = list(t)

In [126]:
r[1][0]

array([[21, 22, 23, 24],
       [26, 27, 28, 29],
       [31, 32, 33, 34],
       [36, 37, 38, 39]])

In [120]:
def create_windowed_dataset(series, window_size,shift_size, batch_size,shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size , shift = shift_size, drop_remainder = True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:,1:],window[window_size - 1,0]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [121]:
df_windowed = create_windowed_dataset(df, 20, 10, 1, 128)

In [122]:
df_windowed

<PrefetchDataset shapes: ((None, None, 3), (None,)), types: (tf.float64, tf.float64)>

In [123]:
l = list(df_windowed.as_numpy_iterator())

In [128]:
features = []
target = []
for i in l:
  print(i)
  print('*************************')
  features.append(i[0])
  target.append(i[1])

(array([[[-0.22053876,  0.26030804, -0.94000127],
        [ 0.53488269,  0.84450583, -0.02665349],
        [-0.43088559,  0.40075373,  0.80853822],
        [-0.05337563,  0.94968346,  0.30862983],
        [ 0.14023914,  0.9836492 ,  0.11299218],
        [ 0.36444961,  0.65847418,  0.65847418],
        [-0.58527729,  0.79734935,  0.14725662],
        [-0.110272  ,  0.67286339,  0.73150185],
        [ 0.65987666, -0.51961302, -0.54273851],
        [ 0.83415229,  0.53690614, -0.12618141],
        [ 0.33484121,  0.94151589, -0.03780465],
        [ 0.21046914,  0.93462569, -0.28666629],
        [ 0.08463995,  0.9202801 ,  0.38199557],
        [-0.42632147,  0.72442595,  0.54171675],
        [-0.03972223,  0.47761254,  0.87767215],
        [-0.53768979,  0.84002714,  0.07241613],
        [ 0.74317007,  0.55328778,  0.37625906],
        [-0.58384127,  0.8035446 ,  0.11595454],
        [ 0.44175539,  0.87418612, -0.20162043],
        [ 0.53268716,  0.84538552, -0.03959428]]]), array([5.]))
***

In [130]:
features = np.array(features)
target = np.array(target)

In [137]:
features =  features.reshape((10,20,3))

In [138]:
features.shape

(10, 20, 3)